In [7]:

import json
import time

PTT_URL = 'https://www.ptt.cc'

In [8]:
import requests
from bs4 import BeautifulSoup


def get_web_page(url):
    resp = requests.get(url=url,
                        cookies={'over18': '1'})
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [9]:
def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html.parser')
    #print(dom)
    # 上頁
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']
    #print('Prev url:',prev_url)

    articles = []
    divs = soup.find_all('div', 'r-ent')
    #print([d.text for d in divs ])
    for d in divs:
        #title=d.find('div',class_='title').text.strip()
        #if '[商品]' in title and d.find('div',class_='date').text.strip==date:
        if d.find('div', class_='date').text.strip() == date:

            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)
                except ValueError:

                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            if d.find('a'):
                href = d.find('a')['href']
                title = d.find('a').text

                author = d.find('div', 'author').text if d.find('div', 'author') else ''
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count,
                    'author': author
                })
    return articles, prev_url

In [10]:
def get_author_ids(posts, pattern):
    ids = set()
    for post in posts:
        if pattern in post['author']:
            ids.add(post['author'])
    return ids


In [11]:
if __name__ == '__main__':
    current_page = get_web_page(PTT_URL + '/bbs/Food/index.html')
    if current_page:
        articles = []
        today = time.strftime("%m/%d").lstrip('0')

        current_articles, prev_url = get_articles(current_page, today)
        while current_articles:
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url = get_articles(current_page, today)

        # print(get_author_ids(articles, '5566'))
        print('今天有', len(articles), '篇文章')
        for a in articles:
            #if int(a['push_count']) > threshold:
            print(a)
        with open('food.json', 'w', encoding='utf-8') as f:
            json.dump(articles, f, indent=2, sort_keys=True, ensure_ascii=False)

今天有 12 篇文章
{'title': '[食記] 彰化透明空氣-田野間的日式絕美餐廳', 'href': '/bbs/Food/M.1648427992.A.F74.html', 'push_count': 0, 'author': 'maybe1230'}
{'title': '[食記] 台南．中西區．錫鼓(窄巷內日式咖啡館)', 'href': '/bbs/Food/M.1648428738.A.3AD.html', 'push_count': 0, 'author': 'lutherwei'}
{'title': '[食記] 台北善導寺站【正宗台南意麵】', 'href': '/bbs/Food/M.1648429654.A.CBE.html', 'push_count': 0, 'author': 'hellolego'}
{'title': '[食記] 台北信義x吳興商圈【一碗小羊肉】', 'href': '/bbs/Food/M.1648429837.A.AB6.html', 'push_count': 0, 'author': 'hellolego'}
{'title': '[食記] 新北 樹林 黑肉圓', 'href': '/bbs/Food/M.1648430402.A.2B7.html', 'push_count': 0, 'author': 'smallta'}
{'title': '[食記] 花蓮市區 平行時空·小茶室', 'href': '/bbs/Food/M.1648430925.A.D8B.html', 'push_count': 0, 'author': 'shangyikuo'}
{'title': '[食記] 高雄三民 小吃 采儀麵館', 'href': '/bbs/Food/M.1648437827.A.DC1.html', 'push_count': 0, 'author': 'BOSOMGIRL'}
{'title': '[食記] 台中市南屯區 廣式養生功夫雞', 'href': '/bbs/Food/M.1648438712.A.D0F.html', 'push_count': 0, 'author': 'grawd'}
{'title': '[食記][東區] 京九食堂｜日式丼飯、海鮮、生食', 'href': '/